In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import deque

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

In [2]:
class DQNagent:

    def __init__(self, state_size, action_size, episodes):
        self.gamma = 0.95
        self.batch_size = 64
        self.state_size = state_size
        self.action_size = action_size
        self.episodes = episodes
        self.replay_memory = deque(maxlen=2000)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.loss_fn = tf.keras.losses.mean_squared_error

    def build_model(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(32, activation="relu", input_shape=[self.state_size]),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(self.action_size)
        ])
        return model

    def add_memory(self, state, action, reward, next_state, done):
        self.replay_memory.append((state, action, reward, next_state, done))

    def sample_experiences(self):
        indices = np.random.randint(len(self.replay_memory), size=self.batch_size)
        batch = [self.replay_memory[index] for index in indices]
        states, actions, rewards, next_states, dones = [
            np.array([experience[field_index] for experience in batch])
            for field_index in range(5)]
        return states, actions, rewards, next_states, dones

    def train_model(self, model):
        states, actions, rewards, next_states, dones = self.sample_experiences()
        next_Q_values = model.predict(next_states)
        max_next_Q_values = np.max(next_Q_values, axis=1)
        target_Q_values = (rewards + (1 - dones) * self.gamma * max_next_Q_values)
        target_Q_values = target_Q_values.reshape(-1, 1)
        mask = tf.one_hot(actions, self.action_size)
        with tf.GradientTape() as tape:
            all_Q_values = model(states)
            Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
            loss = tf.reduce_mean(self.loss_fn(target_Q_values, Q_values))
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [3]:
def render_policy_net(model, n_max_steps=200, seed=42):
    frames = []
    env = gym.make('MountainCar-v0')
    env.seed(seed)
    np.random.seed(seed)
    obs = env.reset()
    for step in range(n_max_steps):
        frames.append(env.render(mode="rgb_array"))
        action = np.argmax(model.predict(obs[np.newaxis])[0])
        obs, reward, done, info = env.step(action)
        if done:
            break
    env.close()
    return frames

def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

In [4]:
best_score = 200
episodes = 2000
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNagent(state_size, action_size, episodes)
model = agent.build_model()
rewards = []

for episode in range(episodes):
    state = env.reset()    
    for step in range(200):
        epsilon = max(1 - episode/(episodes*0.8), 0.01)
        if np.random.rand() < epsilon: action = np.random.randint(action_size)
        else: action = np.argmax(np.array(model.predict(state[np.newaxis])[0]))
        next_state, reward, done, info, _ = env.step(action)
        print(next_state[0], state[0], action)
        if ((next_state[0] - state[0]).any() > 0 & action == 2): reward = 1
        if ((next_state[0] - state[0]).any() < 0 & action == 0): reward = 1        
        agent.add_memory(state, action, reward, next_state, done)
        state = next_state.copy()
        if done:
            break
    rewards.append(step)
    if step < best_score:
        best_weights = model.get_weights()
        best_score = step
    print("\rEpisode: {}, Best Score: {}, eps: {:.3f}".format(episode, best_score, epsilon), end="")
    if episode > 50:
        agent.train_model(model)

model.set_weights(best_weights)
frames = render_policy_net(model)
plot_animation(frames)

env.close()

-0.5828187 [-0.5842712  0.       ] 2
-0.58092433 -0.5828187 1
-0.5796022 -0.58092433 0
-0.5768619 -0.5796022 2
-0.5727239 -0.5768619 2
-0.5692188 -0.5727239 0
-0.5643726 -0.5692188 2
-0.5582214 -0.5643726 2
-0.5518109 -0.5582214 1
-0.5461892 -0.5518109 0
-0.5413981 -0.5461892 0
-0.53747356 -0.5413981 0
-0.532445 -0.53747356 2
-0.5283502 -0.532445 0
-0.52421963 -0.5283502 1
-0.5200845 -0.52421963 1
-0.5169757 -0.5200845 0
-0.51391655 -0.5169757 1
-0.51193 -0.51391655 0
-0.50903094 -0.51193 2
-0.50524116 -0.50903094 2
-0.5025889 -0.50524116 0
-0.5010942 -0.5025889 0
-0.49976814 -0.5010942 1
-0.49962065 -0.49976814 0
-0.49965283 -0.49962065 1
-0.49886444 -0.49965283 2
-0.4972614 -0.49886444 2
-0.4958557 -0.4972614 1
-0.4946578 -0.4958557 1
-0.4926767 -0.4946578 2
-0.4899272 -0.4926767 2
-0.4864298 -0.4899272 2
-0.48421058 -0.4864298 0
-0.48128608 -0.48421058 2
-0.4796781 -0.48128608 0
-0.47939855 -0.4796781 0
-0.47844952 -0.47939855 2
-0.4778381 -0.47844952 1
-0.4785688 -0.4778381 0
-0.48

TypeError: tuple indices must be integers or slices, not NoneType